# 推理引擎ONNX Runtime部署-预测单张图像  

使用推理引擎 ONNX Runtime，读取 ONNX 格式的模型文件，对 **单张图像** 进行预测。

**参考**：[pytorch模型部署](https://space.bilibili.com/1900783)  

要将代码部署到硬件上运行（本地PC、嵌入式开发板、树莓派、Jetson Nano、服务器），

只需把`onnx`模型文件发到待部署的硬件上，并安装 ONNX Runtime 环境，用下面几行代码就可以运行模型了。

In [1]:
import onnxruntime
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd

### （1）**下载素材文件**

包括保存ImageNet类别索引的csv文件、香蕉测试图像banana1.jpg和测试视频video_4.mp4 
 
可直接点击下方链接下载，也可[百度网盘自行下载](https://pan.baidu.com/s/1v9pFvc6-YJ7YZYInK7SrXA)，
提取码：cpdd

In [ ]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/imagenet/imagenet_class_index.csv

!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/test/banana1.jpg
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/test/video_4.mp4

### （2）**加载ONNX模型，获取ONNX Runtime推理器**

In [4]:
ort_session = onnxruntime.InferenceSession("ResNet18_ImageNet.onnx")

**构造随机输入进行测试**

In [8]:
x = torch.randn([1, 3, 256, 256]).numpy()  # 注意转换为numpy格式

# 注意，输入输出张量的名称需要和 torch.onnx.export 中设置的输入输出名对应
ort_input = {"input": x}  # onnx runtime输入
ort_output = ort_session.run(["output"], ort_input)[0]  # onnx runtime输入输出

print(ort_output.shape)

(1, 1000)


### （3）**使用真正的图像进行测试**

In [10]:
from PIL import Image

# 图像路径根据自己的实际进行修改。
image_path = "./素材文件/banana1.jpg"
image = Image.open(image_path)

**测试图像预处理**

In [12]:
from torchvision import transforms

test_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [15]:
input_image = test_transform(image)
print(input_image.shape)  # 输出是tensor类型

# 增加batch_size维度为1，转为numpy格式。
input_tensor = input_image.unsqueeze(0).numpy()
print(input_tensor.shape)

torch.Size([3, 256, 256])
(1, 3, 256, 256)


### （4）**推理预测**

In [16]:
# ONNX Runtime 输入
ort_input = {"input": input_tensor}

# ONNX Runtime 输出
pred_logits = ort_session.run(["output"], ort_input)[0]
pred_logits = torch.tensor(pred_logits)

print(pred_logits.shape)

torch.Size([1, 1000])


In [17]:
# 对 logit 分数做 softmax 运算，得到置信度概率
pred_softmax = F.softmax(pred_logits, dim=1)
print(pred_softmax.shape)

torch.Size([1, 1000])


### （5）**解析预测结果**

In [18]:
# 取置信度最高的前 n 个结果
n = 3

top_n = torch.topk(pred_softmax, n)

print(top_n)

torch.return_types.topk(
values=tensor([[9.9686e-01, 2.4719e-03, 2.8185e-04]]),
indices=tensor([[954, 939, 941]]))


In [20]:
# 预测类别
pred_ids = top_n.indices.numpy()[0]

print("类别是：", pred_ids)

# 预测置信度
confs = top_n.values.numpy()[0]

print("置信度是：", confs)

类别是： [954 939 941]
置信度是： [9.9686396e-01 2.4718815e-03 2.8184804e-04]


### （6）**载入类别 ID 和类别名称 对应关系**

In [22]:
# 路径根据实际进行修改
df = pd.read_csv("./素材文件/imagenet_class_index.csv")

idx_to_labels = {}


for idx, row in df.iterrows():

    # idx_to_labels[row["ID"]] = row["class"]  # 英文

    idx_to_labels[row["ID"]] = row["Chinese"]  # 中文

### （7）**打印预测结果**

In [23]:
for i in range(n):
    class_name = idx_to_labels[pred_ids[i]]  # 获取类别名称
    confidence = confs[i] * 100  # 获取置信度
    text = "{:<20} {:>.3f}".format(class_name, confidence)
    print(text)

香蕉                   99.686
绿皮密生西葫芦              0.247
小青南瓜                 0.028
